# Demo of Inverse Solution

In [1]:
import numpy as np
import numpy.linalg as na

import pandas as pd

import plotly.express as px
import plotly.graph_objects as go

from scipy.optimize import minimize

In [107]:
def deg2arc(deg):
    ''' Convert [deg] into arc '''
    return np.pi / 180 * deg

def arc2deg(arc):
    ''' Convert [arc] into degress '''
    return 180 / np.pi * arc

In [3]:
def rotate(vec, norm, arc):
    ''' Compute the Rotated [vec] with [arc] angles,
        around the direction of [norm].
    '''

    l = na.norm(vec)
    x = vec
    y = np.cross(norm, vec)
    
    if na.norm(y) == 0:
        return x
    
    x = x / na.norm(x) * l * np.cos(arc)
    y = y / na.norm(y) * l * np.sin(arc)
    
    return x + y

In [4]:
def chain(arcs):
    ''' Compute the coordinates of the end point
    '''
    
    pivots = np.array([
        [0, 0, 0],
        [0, 10, 0],
        [0, 18, 0],
        [0, 0, 0]
    ]).astype(np.float32)

    vecs = np.array([
        [0, 10, 0],
        [0, 8, 0],
        [0, 6, 0]
    ]).astype(np.float32)
    
    norms = np.array([
        [0, 1, 0],
        [1, 0, 0],
        [1, 0, 0]
    ]).astype(np.float32)

    # Arm-1
    kwargs = dict(norm=norms[0], arc=arcs[0])

    vecs[0] = rotate(vecs[0], **kwargs)
    vecs[1] = rotate(vecs[1], **kwargs)
    vecs[2] = rotate(vecs[2], **kwargs)
    pivots[1] = pivots[0] + vecs[0]

    norms[1] = rotate(norms[1], **kwargs)
    norms[2] = rotate(norms[2], **kwargs)

    # Arm-2
    kwargs = dict(norm=norms[1], arc=arcs[1])

    vecs[1] = rotate(vecs[1], **kwargs)
    vecs[2] = rotate(vecs[2], **kwargs)
    pivots[2] = pivots[1] + vecs[1]

    norms[2] = rotate(norms[2], **kwargs)

    # Arm-3
    kwargs = dict(norm=norms[2], arc=arcs[2])
    
    vecs[2] = rotate(vecs[2], **kwargs)
    
    pivots[3] = pivots[2] + vecs[2]

    return pivots, vecs, norms

In [86]:
def randomArcs():
    ''' Generate Random Arcs Values '''
    arcs = [
        deg2arc(np.random.randint(0, 360)),
        deg2arc(np.random.randint(0, 90)),
        deg2arc(np.random.randint(-90, 90))
    ]
    return arcs

In [111]:
target = np.array([0, 24, 0]).astype(np.float32)
target = np.array([10, 15, 10]).astype(np.float32)

arcs = randomArcs()
pivots, vecs, norms = chain(arcs)
target = pivots[-1]

target = np.array([0.0, 20, 0])

def fun(x, target=target):
    pivots, vecs, norms = chain(x)
    d = na.norm(pivots[-1] - target)
    return d


cons = (
    dict(type='ineq', fun=lambda x: x[0]),
    dict(type='ineq', fun=lambda x: deg2arc(360) - x[0]),
    dict(type='ineq', fun=lambda x: x[1]),
    dict(type='ineq', fun=lambda x: deg2arc(90) - x[1]),
    dict(type='ineq', fun=lambda x: x[2] + deg2arc(90)),
    dict(type='ineq', fun=lambda x: deg2arc(90) - x[2]),
)

method = 'COBYLA'
res = minimize(fun, randomArcs(), method=method, constraints=cons)
for _ in range(10):
    res1 = minimize(fun, randomArcs(), method=method, constraints=cons)
    if res1.fun < res.fun:
        res = res1
        
# if res.fun > 1e-3:
#     print('Too large, try another')
#     res1 = minimize(fun, x0, method='Powell')
#     if res1.fun < res.fun:
#         res = res1
    
print(res.success, res.fun, res.x, arcs, target)

True 0.00014730439925224586 [ 1.42478848  0.64348524 -0.92730769] [0.4886921905584123, 1.4486232791552935, 0.06981317007977318] [ 0. 20.  0.]


In [112]:
[arc2deg(e) for e in res.x]

[81.63436633464337, 36.86898819708124, -53.13081672308431]

In [113]:
arcs = res.x
pivots, vecs, norms = chain(arcs)

print(pivots)
print(vecs)
print(norms)

df = pd.DataFrame(pivots, columns=['x', 'y', 'z'])
print(df)

fig = px.line_3d(df, x='x', y='y', z='z')
fig.show()

[[ 0.0000000e+00  0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00  1.0000000e+01  0.0000000e+00]
 [ 4.7488265e+00  1.6400076e+01  6.9833547e-01]
 [-1.4495850e-04  2.0000015e+01 -2.1278858e-05]]
[[ 0.         10.          0.        ]
 [ 4.7488265   6.4000764   0.69833547]
 [-4.7489715   3.59994    -0.69835675]]
[[ 0.          1.          0.        ]
 [ 0.14548963  0.         -0.9893598 ]
 [ 0.14548963  0.         -0.9893598 ]]
          x          y         z
0  0.000000   0.000000  0.000000
1  0.000000  10.000000  0.000000
2  4.748827  16.400076  0.698335
3 -0.000145  20.000015 -0.000021


In [114]:
na.norm(pivots[2] - pivots[1])

7.9999995

In [115]:
na.norm(pivots[3] - pivots[2])

5.9999995